In [1]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! mkdir ~/.kaggle

In [3]:
! cp /content/kaggle.json ~/.kaggle/

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
! kaggle datasets download mohamedhanyyy/chest-ctscan-images

100% 119M/119M [00:05<00:00, 29.3MB/s]
100% 119M/119M [00:05<00:00, 24.6MB/s]


In [7]:
!unzip /content/chest-ctscan-images.zip

Archive:  /content/chest-ctscan-images.zip
  inflating: Data/test/adenocarcinoma/000108 (3).png  
  inflating: Data/test/adenocarcinoma/000109 (2).png  
  inflating: Data/test/adenocarcinoma/000109 (4).png  
  inflating: Data/test/adenocarcinoma/000109 (5).png  
  inflating: Data/test/adenocarcinoma/000112 (2).png  
  inflating: Data/test/adenocarcinoma/000113 (7).png  
  inflating: Data/test/adenocarcinoma/000114 (5).png  
  inflating: Data/test/adenocarcinoma/000114.png  
  inflating: Data/test/adenocarcinoma/000115 (4).png  
  inflating: Data/test/adenocarcinoma/000115 (8).png  
  inflating: Data/test/adenocarcinoma/000115.png  
  inflating: Data/test/adenocarcinoma/000116 (5).png  
  inflating: Data/test/adenocarcinoma/000116 (7).png  
  inflating: Data/test/adenocarcinoma/000116 (9).png  
  inflating: Data/test/adenocarcinoma/000117 (4).png  
  inflating: Data/test/adenocarcinoma/000117 (8).png  
  inflating: Data/test/adenocarcinoma/000117.png  
  inflating: Data/test/adenocarcin

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [9]:
ls

chest-ctscan-images.zip  Data/  gdrive/  kaggle.json  sample_data/


In [10]:
pwd

'/content'

In [50]:
import tensorflow as tf
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras import optimizers

In [51]:
train_datagen = ImageDataGenerator(rescale = 1.0/255,rotation_range=10, width_shift_range=0.2, height_shift_range=0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip=True, vertical_flip=True, fill_mode='nearest')
training_set = train_datagen.flow_from_directory('/content/Data/train',
                                                 target_size = (224,224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

Found 613 images belonging to 4 classes.


In [52]:
test_datagen = ImageDataGenerator(rescale = 1.0/255)
test_set = test_datagen.flow_from_directory('/content/Data/valid',
                                                 target_size = (224,224),
                                                 batch_size = 64,
                                                 class_mode = 'categorical')

Found 72 images belonging to 4 classes.


In [ ]:
#cnn = tf.keras.models.Sequential()
#cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224, 224, 3]))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=4, activation='relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=4, activation='relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#cnn.add(tf.keras.layers.Conv2D(filters=256, kernel_size=4, activation='relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#cnn.add(tf.keras.layers.Conv2D(filters=512, kernel_size=4, activation='relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

#cnn.add(tf.keras.layers.Flatten())
#cnn.summary()

#cnn.add(tf.keras.layers.Dense(units=8192, activation='relu'))
#cnn.add(tf.keras.layers.Dropout(0.2))
#cnn.add(tf.keras.layers.Dense(units=4, activation='softmax'))
#cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#cnn.summary()

In [54]:
base_model = tf.keras.applications.VGG16(input_shape=(224,224,3),include_top=False,weights="imagenet")

58889256/58889256 [==============================] - 3s 0us/step


In [55]:
# Freezing Layers
for layer in base_model.layers:
    layer.trainable = False

In [56]:
cnn = base_model.output

In [57]:
cnn

<KerasTensor: shape=(None, 7, 7, 512) dtype=float32 (created by layer 'block5_pool')>

In [58]:
cnn = tf.keras.layers.Flatten()(cnn)
cnn

<KerasTensor: shape=(None, 25088) dtype=float32 (created by layer 'flatten_4')>

In [59]:
cnn = tf.keras.layers.Dense(1024, activation = 'relu')(cnn)
cnn = tf.keras.layers.Dropout(0.2)(cnn)
#cnn = tf.keras.layers.Dense(1024, activation = 'relu')(cnn)
#cnn = tf.keras.layers.Dropout(0.3)(cnn)
cnn = tf.keras.layers.Dense(4, activation = "softmax")(cnn)
model = Model(inputs= base_model.input , outputs = cnn)

In [60]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy',metrics=['accuracy'])

In [61]:
model.fit(x = training_set, validation_data=test_set, epochs = 100)

Epoch 1/100
10/10 [==============================] - 11s 1s/step - loss: 11.3016 - accuracy: 0.3295 - val_loss: 4.6484 - val_accuracy: 0.4444
Epoch 2/100
10/10 [==============================] - 10s 984ms/step - loss: 3.1705 - accuracy: 0.4290 - val_loss: 1.6955 - val_accuracy: 0.3611
Epoch 3/100
10/10 [==============================] - 10s 989ms/step - loss: 1.3972 - accuracy: 0.5122 - val_loss: 1.2705 - val_accuracy: 0.4028
Epoch 4/100
10/10 [==============================] - 10s 1s/step - loss: 1.0507 - accuracy: 0.5693 - val_loss: 1.0588 - val_accuracy: 0.4583
Epoch 5/100
10/10 [==============================] - 12s 1s/step - loss: 0.8863 - accuracy: 0.6101 - val_loss: 0.9112 - val_accuracy: 0.5972
Epoch 6/100
10/10 [==============================] - 10s 979ms/step - loss: 0.8327 - accuracy: 0.6346 - val_loss: 0.9862 - val_accuracy: 0.4861
Epoch 7/100
10/10 [==============================] - 10s 998ms/step - loss: 0.7846 - accuracy: 0.6591 - val_loss: 0.9013 - val_accuracy: 0.5417


In [62]:
model.save('chest_cancer_detector2.h5')

In [ ]:
pwd

'/content'

In [ ]:
#cnn.fit(x = training_set, validation_data=test_set, epochs = 90)

Epoch 1/90
10/10 [==============================] - 166s 16s/step - loss: 1.7053 - accuracy: 0.3148 - val_loss: 1.3208 - val_accuracy: 0.4317
Epoch 2/90
10/10 [==============================] - 155s 16s/step - loss: 1.2291 - accuracy: 0.4372 - val_loss: 1.2325 - val_accuracy: 0.4127
Epoch 3/90
10/10 [==============================] - 154s 15s/step - loss: 1.2753 - accuracy: 0.4078 - val_loss: 1.1180 - val_accuracy: 0.4794
Epoch 4/90
10/10 [==============================] - 159s 16s/step - loss: 1.0673 - accuracy: 0.5237 - val_loss: 1.0080 - val_accuracy: 0.5651
Epoch 5/90
10/10 [==============================] - 152s 15s/step - loss: 1.0051 - accuracy: 0.5122 - val_loss: 1.0111 - val_accuracy: 0.5302
Epoch 6/90
10/10 [==============================] - 155s 16s/step - loss: 1.0403 - accuracy: 0.4927 - val_loss: 0.9305 - val_accuracy: 0.5714
Epoch 7/90
10/10 [==============================] - 154s 15s/step - loss: 0.9769 - accuracy: 0.5090 - val_loss: 1.0210 - val_accuracy: 0.4762
Epoch 

KeyboardInterrupt: ignored